# Knowledge Proficiency Tracing model (KPT) and Exercise-correlated KPT (EKPT)

This notebook will show you how to train and use the KPT and EKPT.
First, we will show how to get the data (here we use assistment-2009-2010-skill as the dataset).
Then we will show how to train a KPT and EKPT, and perform the parameters persistence.
At last, we will show how to load the parameters from the file and evaluate on the test dataset.

The script version could be found in [KPT.py](KPT.ipynb)

## Data Preparation

Before we process the data, we need to first acquire the dataset which is shown in [prepare_dataset.ipynb](prepare_dataset.ipynb)

In [1]:
# Load the data from files
import numpy as np
import json

# Q matrix
q_m = np.loadtxt("../../data/2009_skill_builder_data_corrected/q_m.csv", dtype=int, delimiter=",")
prob_num, know_num = q_m.shape[0], q_m.shape[1]

# training data
with open("../../data/2009_skill_builder_data_corrected/train_data.json", encoding='utf-8') as file:
    train_set = json.load(file)
stu_num = max([x['user_id'] for x in train_set[0]]) + 1
time_window_num = len(train_set)
                    
# testing data
with open("../../data/2009_skill_builder_data_corrected/test_data.json", encoding='utf-8') as file:
    test_set = json.load(file)

In [2]:
# train_set[0][0] is the first train log in the first Time Window
print(train_set[0][0], test_set[0])

{'user_id': 994, 'item_id': 4770, 'score': 0} {'user_id': 21, 'item_id': 5071, 'score': 1}


In [3]:
len(train_set), len(test_set)

(7, 16474)

## Training and Persistence

In [4]:
import logging
logging.getLogger().setLevel(logging.INFO)

### KPT

In [5]:
from EduCDM import KPT

cdm = KPT('KPT', q_m, stu_num, prob_num, know_num, time_window_num=time_window_num)

cdm.train(train_set, epoch=2, lr=0.001, lr_b=0.0001, epsilon=1e-3, init_method='mean')
cdm.save("kpt.params")

INFO:root:save parameters to kpt.params


### EKPT

In [6]:
cdm2 = KPT('EKPT', q_m, stu_num, prob_num, know_num, time_window_num=time_window_num)

cdm2.train(train_set, epoch=2, lr=0.001, lr_b=0.0001, epsilon=1e-3, init_method='mean')
cdm2.save("ekpt.params")

INFO:root:save parameters to ekpt.params


## Loading and Testing

### KPT

In [7]:
cdm.load("kpt.params")
rmse, mae = cdm.eval(test_set)
print("For KPT, RMSE: %.6f, MAE: %.6f" % (rmse, mae))

INFO:root:load parameters from kpt.params
evaluating: 100%|██████████████████████████████████████████████████████████████| 16370/16370 [00:02<00:00, 7099.88it/s]


For KPT, RMSE: 0.445781, MAE: 0.379456


### EKPT

In [8]:
cdm2.load("ekpt.params")
rmse2, mae2 = cdm2.eval(test_set)
print("For EKPT, RMSE: %.6f, MAE: %.6f" % (rmse2, mae2))

INFO:root:load parameters from ekpt.params
evaluating: 100%|██████████████████████████████████████████████████████████████| 16370/16370 [00:02<00:00, 7576.03it/s]


For EKPT, RMSE: 0.446698, MAE: 0.379953
